# IS-BQPhy-Optimization-Proj-F25 Notebook

In [90]:
from typing import List, Tuple

import numpy as np
import bqphy.BQPhy_Optimiser as qea

In [91]:
# ------------------------ PROBLEM-SPECIFIC: Portfolio Data ------------------------ #
# --- MOCK DATA (PROBLEM-SPECIFIC) ---

# Expected returns per $1 invested (decimal)
arrExpectedReturns = np.array([
    0.12, 0.08, 0.15, 0.09, 0.11, 0.13, 0.07, 0.14, 0.10, 0.09,
    0.16, 0.08, 0.12, 0.11, 0.13, 0.10, 0.09, 0.14, 0.15, 0.12,
    0.08, 0.13, 0.11, 0.07, 0.10
], dtype=float)

# Standard deviations (volatility) per asset (PROBLEM-SPECIFIC)
arrVolatilities = np.array([
    0.2, 0.25, 0.18, 0.22, 0.19, 0.21, 0.16, 0.23, 0.20, 0.18,
    0.24, 0.19, 0.22, 0.20, 0.21, 0.18, 0.17, 0.23, 0.25, 0.20,
    0.16, 0.21, 0.19, 0.15, 0.20
], dtype=float)

# Price per share (PROBLEM-SPECIFIC)
arrPrices = np.array([
    50, 100, 80, 120, 60, 90, 40, 70, 110, 85,
    95, 55, 65, 75, 105, 60, 80, 90, 100, 50,
    45, 85, 70, 60, 95
], dtype=float)

NUM_ASSETS = arrExpectedReturns.shape[0]

In [92]:
# Preferences / constraints (PROBLEM-SPECIFIC)
availableCapital = 1000.0          # hard capital limit
preferred_min_stocks = 5          # soft diversification lower bound
preferred_max_stocks = 15         # soft diversification upper bound

In [93]:
# Objective/Fitness function parameters (PROBLEM-SPECIFIC / tunable)
lambda_risk = 1.0                 # risk aversion multiplier
hard_violation_penalty = 1e6      # very large to enforce hard constraints
under_diversification_penalty = 10.0         # penalty per missing stock if under-diversified
over_diversification_penalty = 5.0           # penalty per extra stock if over-diversified

In [94]:
# ------------------------ FITNESS FUNCTION (vectorized) ------------------------ #
def portfolio_fitness(x):
  """
  Vectorized fitness function for BQPhy optimizer.
  Accepts:
  - 1D numpy array of length NUM_ASSETS (single candidate)
  - OR 2D numpy array shape (population_size, NUM_ASSETS)

  Returns:
  - scalar fitness (if input 1D) or 1D array of fitness values (if input 2D)
  Fitness is minimized by BQPhy.
  """

  # ------------------------  Data Preprocessing ------------------------ #

  # Normalize input to 2D for unified processing
  arr = np.array(x, dtype=float)
  single = False
  if arr.ndim == 1:
    arr = arr.reshape(1, -1)
    single = True

  # Ensure binary inputs (optimizer will produce binary when config set to BINARY).
  bin_arr = (arr >= 0.5).astype(float)  # shape (pop, NUM_ASSETS)

  # ------------------------  Objective Function ------------------------ #

  # Calculate objective function components
  expected_returns = bin_arr @ arrExpectedReturns
  variances = bin_arr @ (arrVolatilities ** 2)  # Using diagonal approximation
  total_investments = bin_arr @ arrPrices     
  num_selected = np.sum(bin_arr > 0.0, axis=1)

  # Caulcate objective function values
  objective_vals = expected_returns - lambda_risk * variances  

  # ------------------------ HARD CONSTRAINTS ------------------------ #

  # HC1 - Impossible to invest more than you have
  violation1 = np.maximum(total_investments - availableCapital, 0.0)

  # HC2 - Must invest something
  violation2 = (total_investments <= 0.0)

  # HC3 - No shorting
  violation3 = np.maximum(-np.minimum(arr, 0.0).sum(axis=1), 0.0)  # sum negative parts

  # ------------------------ SOFT CONSTRAINTS ------------------------ #

  # SC1 - Penalize Insufficient Diversification
  soft_penalty1 = np.maximum(preferred_min_stocks - num_selected, 0.0)

  # SC2 - Penalize Over-Diversification
  soft_penalty2 = np.maximum(num_selected - preferred_max_stocks, 0.0)

  # ------------------------ FITNESS FUNCTION ------------------------ #
  fitness = -objective_vals.copy()
  fitness += violation1.astype(float) * hard_violation_penalty
  fitness += violation2.astype(float) * hard_violation_penalty
  fitness += violation3.astype(float) * hard_violation_penalty
  fitness += soft_penalty1 * under_diversification_penalty
  fitness += soft_penalty2 * over_diversification_penalty

  # If single input, return scalar
  if single:
    return float(fitness[0])
  return fitness  # numpy array


In [104]:
# ------------------------ HELPER: display solution ------------------------ #
def display_solution(solution: List[int]) -> Tuple[float, float, float]:
    """Compute portfolio return, volatility, and print clean summary."""

    sol = np.array(solution, dtype=int)
    selected_idx = list(np.where(sol == 1)[0])

    # Edge case: nothing selected
    if not selected_idx:
        print("\nNo assets selected.")
        return 0.0, 0.0, 0.0

    total_investment = float(np.sum(arrPrices[selected_idx]))
    weights = np.array([arrPrices[i] for i in selected_idx]) / total_investment
    portfolio_return = float(
        np.sum(weights * np.array([arrExpectedReturns[i] for i in selected_idx]))
    )
    portfolio_variance = float(
        np.sum((weights ** 2) * (np.array([arrVolatilities[i] for i in selected_idx]) ** 2))
    )
    portfolio_volatility = portfolio_variance ** 0.5

    # ----------- PRINT CLEAN SUMMARY -----------
    print(f"\n--------------------- Portfolio Summary ---------------------")
    # print(f"Selected Assets: {selected_idx}") # Will Need When Transformed to Real Data
    print(f"Total Investment: ${total_investment:.2f} / ${availableCapital:.2f}")
    print(f"Portfolio Expected Return: {portfolio_return * 100:.2f}%")
    print(f"Portfolio Volatility (est.): {portfolio_volatility * 100:.2f}%")
    print(f"Number of Selected Assets: {len(selected_idx)}")

    feasible = total_investment <= availableCapital
    print(f"Feasible: {'Yes' if feasible else 'No'}")

    return portfolio_return, total_investment, portfolio_volatility

In [105]:
def main():
    print("BQPhy Binary Portfolio Optimization")
    print("" + "=" * 60)

    # Data Summary
    print("Input Data Summary\n")
    for i in range(NUM_ASSETS):
        print(f" Asset {i:2d}: price={arrPrices[i]:3.0f}, expReturn={arrExpectedReturns[i]:.3f}, vol={arrVolatilities[i]:.3f}")
    print(f"\nAvailable capital: ${availableCapital:.2f}")
    print(f"Preferred diversification: {preferred_min_stocks} - {preferred_max_stocks} assets\n")
    print("" + "=" * 60)

    # BQPhy configuration
    config = {
        "numPopulation": 100, # tunable
        "maxGeneration": 200, # tunable
        "deltaTheta": .05, # tunable
        "designVariables": NUM_ASSETS, # based on mock data, fixed relative config
        "typeOfOptimisation": "BINARY" # not tunable, fixed config
    }

    # Initialize and configure optimizer
    optimizer = qea.BQPhy_OPTIMISER()
    optimizer.initialize(config)

    # Register model (fitness function)
    optimizer.model(portfolio_fitness)

    # Run optimization (backend)
    print("\n🔄 Running optimizer (openmp backend)...")
    optimizer.runOptimization("openmp")

    # Retrieve best design
    best_solution, best_fitness = optimizer.getBestDesign()

    # All Summary Statistics
    print("\n" + "=" * 60)
    print("   Optimization Complete\n")
    print(f"   Best Fitness (minimized): {best_fitness}")
    print(f"   Best Solution (binary vector): {best_solution}")
    portfolio_return, total_investment, portfolio_volatility = display_solution(best_solution)
    print("" + "-" * 62)


In [106]:
main()

BQPhy Binary Portfolio Optimization
Input Data Summary

 Asset  0: price= 50, expReturn=0.120, vol=0.200
 Asset  1: price=100, expReturn=0.080, vol=0.250
 Asset  2: price= 80, expReturn=0.150, vol=0.180
 Asset  3: price=120, expReturn=0.090, vol=0.220
 Asset  4: price= 60, expReturn=0.110, vol=0.190
 Asset  5: price= 90, expReturn=0.130, vol=0.210
 Asset  6: price= 40, expReturn=0.070, vol=0.160
 Asset  7: price= 70, expReturn=0.140, vol=0.230
 Asset  8: price=110, expReturn=0.100, vol=0.200
 Asset  9: price= 85, expReturn=0.090, vol=0.180
 Asset 10: price= 95, expReturn=0.160, vol=0.240
 Asset 11: price= 55, expReturn=0.080, vol=0.190
 Asset 12: price= 65, expReturn=0.120, vol=0.220
 Asset 13: price= 75, expReturn=0.110, vol=0.200
 Asset 14: price=105, expReturn=0.130, vol=0.210
 Asset 15: price= 60, expReturn=0.100, vol=0.180
 Asset 16: price= 80, expReturn=0.090, vol=0.170
 Asset 17: price= 90, expReturn=0.140, vol=0.230
 Asset 18: price=100, expReturn=0.150, vol=0.250
 Asset 19: pr